# Continual Pre-Training (CPT) – Syarat dan Ketentuan wondr by BNI
---

## Latar Belakang
Perkembangan *Large Language Model* (LLM) mendorong kebutuhan adaptasi model terhadap domain spesifik, khususnya pada konteks perbankan dan layanan keuangan. Salah satu pendekatan yang umum digunakan adalah **Continual Pre-Training (CPT)**, yaitu proses pelatihan lanjutan model bahasa menggunakan data domain-spesifik tanpa menghilangkan kemampuan bahasa umum yang telah dimiliki sebelumnya.

Dokumen **Syarat dan Ketentuan wondr by BNI** merupakan sumber data tekstual yang kaya akan terminologi hukum, operasional perbankan, serta ketentuan layanan digital. Konten ini relevan untuk meningkatkan pemahaman model terhadap konteks regulasi, definisi layanan, dan aturan penggunaan aplikasi perbankan digital. Anda dapat mengunjungi situs resmi wondr by BNI [di sini](https://wondr.bni.co.id/info/terms-conditions).

## Tujuan CPT
Proses CPT ini bertujuan untuk:
- Meningkatkan pemahaman model terhadap bahasa hukum dan regulasi perbankan
- Mengadaptasi model agar lebih akurat dalam memahami konteks layanan *wondr by BNI*
- Mengurangi *hallucination* pada pertanyaan terkait syarat, ketentuan, dan definisi layanan
- Menyediakan fondasi model yang lebih baik untuk tahap *Instruction Fine-Tuning (IFT)* atau *Retrieval-Augmented Generation (RAG)*

## Ruang Lingkup Data
Data yang digunakan dalam CPT ini bersumber dari:
- Dokumen resmi **Syarat dan Ketentuan wondr by BNI**
- Setiap section dan sub-section diperlakukan sebagai satu unit teks mandiri
- Data telah melalui proses pembersihan (*cleaning*) dan normalisasi teks

## Catatan Penting
CPT ini **tidak dimaksudkan untuk menggantikan dokumen resmi** dan hanya digunakan untuk tujuan pengembangan serta eksperimen model bahasa. Interpretasi hukum tetap mengacu pada kebijakan resmi yang diterbitkan oleh Bank.

---

## Table of Content

1. [Pendahuluan](#pendahuluan)
   - [Latar Belakang](#latar-belakang)
   - [Tujuan CPT](#tujuan-cpt)
   - [Ruang Lingkup Data](#ruang-lingkup-data)
   - [Catatan Penting](#catatan-penting)

2. [Deskripsi Data](#deskripsi-data)
   - [Sumber Data](#sumber-data)
   - [Struktur Dokumen](#struktur-dokumen)

3. [Pra-pemrosesan Data](#pra-pemrosesan-data)
   - [Pembersihan Teks](#pembersihan-teks)
   - [Segmentasi Section](#segmentasi-section)
   - [Normalisasi dan Formatting](#normalisasi-dan-formatting)

4. [Metodologi Continual Pre-Training](#metodologi-continual-pre-training)
   - [Arsitektur Model](#arsitektur-model)
   - [Strategi CPT](#strategi-cpt)
   - [Konfigurasi Hyperparameter](#konfigurasi-hyperparameter)

5. [Implementasi](#implementasi)
   - [Pipeline Training](#pipeline-training)
   - [Manajemen Checkpoint](#manajemen-checkpoint)
   - [Kriteria Stopping](#kriteria-stopping)

6. [Evaluasi](#evaluasi)
   - [Metode Evaluasi](#metode-evaluasi)
   - [Contoh Prompt dan Output](#contoh-prompt-dan-output)

7. [Risiko dan Keterbatasan](#risiko-dan-keterbatasan)

8. [Kesimpulan](#kesimpulan)

## Deskripsi Data
---

### Sumber Data

Data yang digunakan dalam proses **Continual Pre-Training (CPT)** ini bersumber dari [**website resmi wondr by BNI**](https://wondr.bni.co.id/). Sumber utama data adalah dokumen **Syarat dan Ketentuan Aplikasi wondr by BNI** yang dipublikasikan secara terbuka dan diperuntukkan bagi nasabah sebagai acuan resmi penggunaan layanan.

Konten yang diambil mencakup antara lain:
- Definisi istilah penting dalam ekosistem wondr by BNI
- Terminologi produk perbankan (tabungan, deposito, kartu, transaksi)
- Ketentuan keamanan dan autentikasi (PIN, Password, OTP)
- Ketentuan penggunaan aplikasi dan keanggotaan
- Informasi terkait data pribadi dan privasi nasabah
- Ketentuan operasional transaksi finansial dan non-finansial

Data diambil langsung dari halaman web resmi tanpa modifikasi makna, kemudian diproses secara teknis untuk kebutuhan pelatihan model bahasa. Penggunaan data ini ditujukan **semata-mata untuk keperluan pengembangan dan eksperimen model**, bukan sebagai pengganti dokumen hukum resmi.

### Struktur Dokumen

Setiap poin, definisi, maupun ketentuan yang tercantum dalam dokumen Syarat dan Ketentuan wondr by BNI diperlakukan sebagai satu unit data pelatihan (*training sample*) yang berdiri sendiri dan digunakan dalam proses Continual Pre-Training pada model Large Language Model (LLM).

## Pra-pemrosesan Data
---

### Import Library

In [ ]:
# install library yang diperlukan
!pip install -q transformers datasets torch accelerate

In [ ]:
# import library untuk melakukan CPT
import torch
import json
import math
import warnings

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import Dataset
warnings.filterwarnings("ignore")

In [ ]:
# mendefinisikan foundational model (base model) yang akan digunakan untuk CPT
model_name = "Qwen/Qwen2.5-0.5B"

# load tokenizer yang sesuai dengan model pretrained
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# load model untuk text generation
model = AutoModelForCausalLM.from_pretrained(model_name)

### Normalisasi dan Formatting

In [ ]:
# initial load dataset untuk proses CPT -> S&K Wondr by BNI
data_sk_bni = {
    "text": [
        text.strip() + tokenizer.eos_token
        for text in [
            "Aplikasi wondr by BNI adalah fasilitas elektronik channel yang dimiliki, dikembangkan, dan dikelola oleh Bank serta dapat diakses dan digunakan oleh Pengguna untuk melakukan Transaksi Finansial dan/atau Transaksi Non Finansial melalui Smartphone.",
            "Bank adalah PT Bank Negara Indonesia (Persero) Tbk yang meliputi kantor pusat dan kantor cabang serta kantor lainnya yang merupakan bagian yang tidak terpisahkan dari PT Bank Negara Indonesia (Persero) Tbk.",      
            "BNI Call Officer (BCO) adalah petugas-petugas dari BNI Customer Experience Center sebagai penerima komplain yang disampaikan oleh Pengguna.",        
            "BNI Call adalah layanan 24 jam yang dapat diakses Nasabah dengan mudah dalam berinteraksi dengan Bank melalui saluran telepon 1500046.",        
            "BNI Deposito adalah simpanan berjangka yang dibuka melalui Aplikasi wondr by BNI dan pencairannya hanya dapat dilakukan pada waktu tertentu sesuai dengan kesepakatan antara Pengguna dan Bank.",        
            "Taplus/Taplus Bisnis/Taplus Muda/Taplus Diaspora/Emerald Saving/BNI Dollar USD/BNI Dollar SGD/BNI Dollar AUD (“Rekening Tabungan”) adalah produk tabungan yang dibuka melalui Aplikasi wondr by BNI yang penarikannya dapat dilakukan setiap saat menurut syarat tertentu yang disepakati antara Pengguna dengan Bank.",        
            "Taplus Digital adalah produk tabungan yang dibuka melalui Aplikasi wondr by BNI yang penarikannya dapat dilakukan setiap saat menurut syarat tertentu yang disepakati antara Nasabah dengan BNI.",        
            "Bunga BNI Deposito adalah bunga yang akan dibayarkan dengan pilihan ditransfer ke Rekening Afiliasi atau kapitalisasi (menambah pokok simpanan BNI Deposito) sampai dengan tanggal jatuh tempo untuk BNI Deposito.",        
            "Data Biometrik adalah yaitu data yang berkaitan dengan fisik, fisiologis, atau karakteristik perilaku individu yang memungkinkan identifikasi unik terhadap individu, termasuk, tetapi tidak terbatas pada, gambar wajah, sidik jari, dan rekaman pembicaraan.",        
            "Data Pribadi adalah data tentang orang perseorangan yang teridentifikasi atau dapat diidentifikasi secara tersendiri atau dikombinasi dengan informasi lainnya baik secara langsung maupun tidak langsung melalui sistem elektronik atau non elektronik.",        
            "Kartu Debit BNI adalah kartu yang diterbitkan oleh Bank berdasarkan lisensi dari principal yang dapat dipergunakan sebagai alat tarik tunai dan melakukan transaksi belanja di jaringan merchant dengan pembebanan langsung ke rekening tabungan atau giro Pengguna sesuai nilai transaksinya.",        
            "Pemberitahuan Privasi adalah bentuk keterbukaan informasi dari Bank yang menjelaskan kepada Nasabah sehubungan dengan Bank memperoleh dan mengumpulkan, mengolah, memproses, menganalisis, menggunakan, menyimpan, memperbaiki dan memperbarui, menampilkan, mengumumkan, mentransfer, menyebarluaskan, mengungkapkan, menghapuskan dan/atau memusnahkan Data Pribadi Nasabah, serta hak-hak Nasabah selaku subjek Data Pribadi berdasarkan ketentuan Undang-Undang Nomor 27 Tahun 2022 tentang Pelindungan Data Pribadi beserta perubahan dan peraturan pelaksanaannya dan peraturan perundang-undangan lain yang berlaku dan relevan.",        
            "Kode One Time Password (selanjutnya disebut “Kode OTP”) adalah kode otentikasi sekali pakai yang dikirimkan melalui SMS/aplikasi WhatsApp kepada Pengguna untuk validasi proses Registrasi Aplikasi wondr by BNI / Reaktivasi Aplikasi wondr by BNI. Kode OTP dikirim oleh sistem Aplikasi wondr by BNI ke nomor Smartphone Pengguna yang telah terdaftar/tercatat pada sistem Bank pada saat Pengguna melakukan Registrasi Aplikasi wondr by BNI / Reaktivasi Aplikasi wondr by BNI.",        
            "Media Resmi Bank adalah sarana penyampaian informasi resmi dari Bank kepada Nasabah antara lain website/SMS/WhatsApp/Surat Elektronik/BNI Call/surat/publikasi resmi di media massa.",        
            "PIN wondr adalah kode untuk otorisasi transaksi, terdiri dari 6 (enam) karakter angka yang bersifat rahasia (hanya boleh diketahui oleh Pengguna yang sah), yang dibuat sendiri oleh Nasabah pada saat melakukan Registrasi atau Reaktivasi, dan dapat diubah sewaktu-waktu oleh Nasabah.",        
            "Nasabah adalah orang perorangan pemilik/pengguna produk perbankan yang disediakan oleh Bank, seperti dan tidak terbatas pada produk tabungan, giro perorangan, deposito dan/atau kartu kredit.",        
            "Nasabah Pengguna Aplikasi wondr by BNI (selanjutnya disebut “Pengguna”) adalah Nasabah yang telah terdaftar sebagai pengguna Aplikasi wondr by BNI.",        
            "Notifikasi Penempatan Dana BNI Deposito/Pencairan BNI Deposito/perubahan Rekening Afiliasi adalah pemberitahuan bahwa proses Penempatan Dana BNI Deposito/Pencairan BNI Deposito/perubahan Rekening Afiliasi berhasil dilakukan oleh Pengguna.",        
            "Password Login adalah kata sandi untuk akses atau login Aplikasi wondr by BNI, terdiri dari 6-12 karakter kombinasi huruf besar dan kecil, angka serta karakter spesial (kecuali ; - + ‘ \\ () = ><@ dan spasi) yang bersifat rahasia, hanya boleh diketahui oleh Pengguna yang sah, yang dibuat sendiri oleh Pengguna di Aplikasi wondr by BNI dan dapat diubah sewaktu-waktu oleh Pengguna melalui Aplikasi wondr by BNI.",        
            "Pencairan BNI Deposito adalah proses pemindahan dana dari rekening BNI Deposito ke Rekening Afiliasi melalui Aplikasi wondr by BNI pada saat tanggal jatuh tempo maupun sebelum tanggal jatuh tempo.",        
            "Penempatan Dana BNI Deposito adalah proses penempatan dana dari Rekening Sumber Dana ke rekening BNI Deposito melalui Aplikasi wondr by BNI.",        
            "Quick Response Code Indonesian Standard (QRIS) adalah Standard QR Code pembayaran yang ditetapkan oleh Bank Indonesia untuk digunakan dalam memfasilitasi pembayaran di Indonesia.",        
            "Reaktivasi adalah proses pengaktifan kembali akun Pengguna agar dapat menggunakan Aplikasi wondr by BNI beserta seluruh transaksi dan/atau fitur yang terdapat di Aplikasi wondr by BNI, dengan proses verifikasi ulang 2 (dua) faktor autentikasi Pengguna terhadap 1 (satu) Smartphone.",        
            "Registrasi adalah proses pembuatan akun Pengguna di 1 (satu) Smartphone yang telah berhasil melakukan instalasi Aplikasi wondr by BNI dan verifikasi 2 (dua) faktor autentikasi.",        
            "Rekening adalah catatan pembukuan Bank atas produk simpanan yang dibuka oleh Pengguna pada Bank melalui Aplikasi wondr by BNI, berupa tabungan dan/atau Deposito, baik dalam Rupiah maupun mata uang asing, atas dasar permohonan pembukaan rekening dari Pengguna, menurut tata cara dan persyaratan yang tercantum dalam Syarat dan Ketentuan ini.",        
            "Rekening Afiliasi adalah rekening simpanan yang ada pada Bank termasuk namun tidak terbatas pada BNI Taplus/BNI Taplus Bisnis/BNI Giro/BNI Emerald Saving yang digunakan Nasabah untuk menampung pembayaran/penerimaan dana seperti penerimaan Bunga BNI Deposito dan pencairan BNI Deposito.",        
            "Rekening Sumber Dana adalah rekening yang terhubung dengan Aplikasi wondr by BNI yang digunakan sebagai rekening pendebetan setoran awal, termasuk namun tidak terbatas pada BNI Taplus/BNI Taplus Bisnis/BNI Giro Perorangan/BNI Taplus Muda/BNI Taplus Diaspora.",        
            "Rekening Sumber Dana BNI Deposito adalah rekening BNI Taplus/BNI Taplus Bisnis/BNI Giro/BNI Emerald Saving/BNI Dollar USD/BNI Dollar SGD/BNI Dollar AUD yang digunakan Pengguna sebagai sumber dana untuk Penempatan Dana BNI Deposito.",        
            "Smartphone adalah handphone dengan sistem operasi Android dan iOS yang memiliki kemampuan tingkat tinggi, yang bekerja menggunakan seluruh perangkat lunak sistem operasi yang menyediakan hubungan standar dan mendasar bagi pengembang aplikasi.",        
            "Service Level Agreement Penyelesaian Komplain (selanjutnya disebut “SLA Penyelesaian Komplain”) adalah acuan dasar penyelesaian komplain di bidang perbankan yang telah ditentukan pihak regulator (dhi. Bank Indonesia dan Otoritas Jasa Keuangan).",       
            "Transaksi Finansial adalah jenis transaksi yang mengakibatkan terjadinya mutasi secara finansial (debet atau kredit) terhadap rekening.",        
            "Transaksi Non Finansial adalah jenis transaksi yang tidak mengakibatkan terjadinya mutasi secara finansial terhadap rekening.",        
            "Transaksi PIN-Less adalah fitur yang disediakan Bank untuk memudahkan Pengguna melakukan transaksi tertentu tanpa perlu memasukan PIN wondr.",        
            "Transaksi Quick Access adalah fitur yang disediakan Bank untuk memudahkan Pengguna dalam melakukan transaksi pembayaran dan/atau pembelian tanpa melakukan login di Aplikasi wondr by BNI.",        
            "Transfer Terjadwal adalah proses pengiriman uang (transfer) yang dijalankan secara otomatis oleh system pada hari/tanggal tertentu sebesar nominal yang ditetapkan oleh Nasabah saat pendaftaran transfer terjadwal.",
            "Ketentuan umum keanggotaan atas Aplikasi wondr by BNI diatur sebagai berikut: \n- Untuk dapat menggunakan Aplikasi wondr by BNI, calon Pengguna harus memenuhi persyaratan Pengguna \n- Pengguna wajib melakukan pengkinian data ke Bank setiap kali ada perubahan Data Pribadi khususnya perubahan alamat e-mail dan/atau nomor Smartphone yang aktif digunakan \n- Segala kerugian yang timbul akibat tidak dilakukannya pengkinian data oleh Pengguna kepada Bank, khususnya perubahan alamat e-mail dan/atau nomor Smartphone aktif Pengguna, menjadi tanggung jawab Pengguna sepenuhnya."
        ]
    ] 
}

In [ ]:
def tokenize_function(examples):
    """
    Melakukan tokenisasi teks untuk kebutuhan Continual Pre-Training (CPT)
    pada model Causal Language Model (CLM).

    Setiap teks akan:
    - Ditokenisasi tanpa menambahkan special token bawaan tokenizer
    - Dipotong hingga panjang maksimum (max_length - 1)
    - Ditambahkan token EOS di akhir sebagai penanda akhir urutan
    - Dibuatkan attention mask bernilai 1 untuk seluruh token aktual

    Args:
        examples (dict): Batch data dari Hugging Face Dataset
            dengan key "text" yang berisi list string teks.

    Returns:
        dict: Dictionary berisi:
            - "input_ids" (List[List[int]]): Token ID hasil tokenisasi
            - "attention_mask" (List[List[int]]): Mask perhatian untuk input_ids
    """
    input_ids = []
    attention_masks = []

    for text in examples["text"]:
        ids = tokenizer.encode(text, add_special_tokens=False)

        # sisakan 1 slot untuk EOS
        max_len = 128 - 1
        ids = ids[:max_len]

        # tambahkan EOS token ASLI
        ids.append(tokenizer.eos_token_id)

        input_ids.append(ids)
        attention_masks.append([1] * len(ids))

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks
    }

In [ ]:
# mengonversi dictionary Python menjadi Hugging Face Dataset
dataset = Dataset.from_dict(data_sk_bni)

# Menyiapkan dataset ter-tokenisasi dan data collator untuk pelatihan Causal Language Model (CPT)
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Menyiapkan data collator untuk batching, padding dinamis, dan pembuatan label pada Causal Language Model
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

## Metodologi Continual Pre-Training (CPT)
---

### Arsitektur Model

### Strategi CPT

### Konfigurasi Hyperparameter

In [ ]:
training_args = TrainingArguments(
    output_dir="./smollm-cpt-results",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10,
    logging_steps=1,         # Tampilkan progress loss setiap berapa step
    learning_rate=2e-5,      # Biasanya gunakan rate kecil untuk mencegah catastrophic forgetting
    weight_decay=0.01,
    fp16=torch.cuda.is_available(), # gunakan mixed precision jika GPU available
    report_to="none", # jangan pakai wandb
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

In [ ]:
# Preplexity untuk mengevaluasi apakah model sebuah mampu "menghafal" tulisan tersebut
def evaluate_perplexity(model, dataset, tokenizer):
    model.eval()
    total_loss = 0
    
    # small batch size
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, collate_fn=data_collator)
    
    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(model.device) for k, v in batch.items()}
            outputs = model(**inputs)
            # Model secara otomatis menghitung CrossEntropyLoss ketika Labels diberikan
            loss = outputs.loss
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    perplexity = math.exp(avg_loss)
    return perplexity

In [ ]:
ppl_before = evaluate_perplexity(model, tokenized_dataset, tokenizer)
print(f"Perplexity before CPT: {ppl_before:.2f}")

trainer.train()
trainer.save_model("./smollm-adapted")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_path = "./smollm-adapted"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

ppl_after = evaluate_perplexity(model, tokenized_dataset, tokenizer)
print(f"Perplexity after CPT: {ppl_after:.2f}")

print(f"Improvement: {ppl_before - ppl_after:.2f} points")

In [ ]:
newline_token_ids = tokenizer.encode("\n", add_special_tokens=False)
print(newline_token_ids)

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StopOnPeriod(StoppingCriteria):
    def __init__(self, dot_token_ids):
        self.dot_token_ids = dot_token_ids

    def __call__(self, input_ids, scores, **kwargs):
        last_token = input_ids[0, -1]
        return last_token in self.dot_token_ids

In [ ]:
def generate_text(prompt, max_new_tokens=150):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    dot_token_ids = tokenizer.encode(".", add_special_tokens=False)

    stopping_criteria = StoppingCriteriaList([
        StopOnPeriod(dot_token_ids)
    ])

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.1,
        top_p=0.3,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        stopping_criteria=stopping_criteria,
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=False)

In [ ]:
prompts = [
    # "BNI Call adalah",
    # "Reaktivasi akun adalah",
    # "Kode OTP",
    # "Transfer Terjadwal",
    "Apa ketentuan umum keanggotaan wondr by bni"
]

for p in prompts:
    result = generate_text(p)
    print(f"Prompt: {p}")
    print(f"Response: {result}")